In [41]:
import re
import json
from pathlib import Path
from jinja2 import Environment, FileSystemLoader

# Function to extract `check` statements from route files
def extract_checks_from_routes(folder_path):
    route_data = {}
    route_regex = re.compile(r"routes\.(?:get|post|put|delete|patch)\(['\"](.*?)['\"],\s*\[(.*?)\]", re.DOTALL)
    check_regex = re.compile(r"check\(['\"](.*?)['\"].*?\)\.(is\w+)")
    body_regex = re.compile(r"body\(['\"](.*?)['\"].*?\)\.(is\w+)")
    query_regex = re.compile(r"query\(['\"](.*?)['\"].*?\)\.(is\w+)")
    param_regex = re.compile(r"param\(['\"](.*?)['\"].*?\)\.(is\w+)")
    cookie_regex = re.compile(r"cookie\(['\"](.*?)['\"].*?\)\.(is\w+)")
    header_regex = re.compile(r"header\(['\"](.*?)['\"].*?\)\.(is\w+)")
    custom_regex = re.compile(r"custom\(['\"](.*?)['\"].*?\)\.(is\w+)")

    for file_path in Path(folder_path).rglob('index.js'):
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            for route_match in route_regex.finditer(content):
                route_path = route_match.group(1)
                checks_block = route_match.group(2)
                
                checks = []
                regex_pairs = [
                    (check_regex, 'check'),
                    (body_regex, 'body'),
                    (query_regex, 'query'),
                    (param_regex, 'param'),
                    (cookie_regex, 'cookie'),
                    (header_regex, 'header'),
                    (custom_regex, 'custom')
                ]

                for regex, source in regex_pairs:
                    for match in regex.finditer(checks_block):
                        param = match.group(1)
                        data_type = match.group(2)
                        checks.append({
                            'param': param, 
                            'type': data_type,
                            'source': source
                        })
                
                route_data[route_path] = checks
    return route_data


# Function to replace markdown links with HTML links
def replace_markdown_links_exclude_tab_content(text):
    # Pattern to match <div class="tab-content">...</div>
    tab_content_pattern = r'<div class="tab-content">.*?</div>'
    # Extract tab-content blocks
    tab_content_blocks = re.findall(tab_content_pattern, text, flags=re.DOTALL)

    # Temporarily replace <div class="tab-content"> blocks with placeholders
    placeholders = {}
    for i, block in enumerate(tab_content_blocks):
        placeholder = f"__TAB_CONTENT_{i}__"
        placeholders[placeholder] = block
        text = text.replace(block, placeholder)

    # Replace markdown links outside of <div class="tab-content">
    markdown_pattern = r'\[([^\]]+)]\((https?:\/\/[^\)]+)\)'
    text = re.sub(markdown_pattern, r'<a href="\2" target="_blank">\1</a>', text)

    # Restore <div class="tab-content"> blocks
    for placeholder, block in placeholders.items():
        text = text.replace(placeholder, block)

    return text

# Preprocess JSON data to replace markdown links
def preprocess_data(data):
    if isinstance(data, dict):
        return {key: preprocess_data(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [preprocess_data(item) for item in data]
    elif isinstance(data, str):
        return replace_markdown_links_exclude_tab_content(data)
    else:
        return data

# Function to find parameters for a given URL
def find_parameters(url, req_params):
    for route, params in req_params.items():
        if route in url:
            return params
    return []

# Main function to integrate everything
def main():
    # Paths
    routes_folder_path = "routes"  # Replace with your routes folder path
    postman_input_path = "json files/input.json"  # Postman export file
    output_html_path = "index.html"  # Output HTML file
    template_path = "templates/template.html"  # Template file

    # Step 1: Extract route checks and parameters
    req_params = extract_checks_from_routes(routes_folder_path)

    # Step 2: Load Postman export JSON
    with open(postman_input_path, 'r', encoding='utf-8') as json_file:
        postman_data = json.load(json_file)
    postman_data = preprocess_data(postman_data)

    # Step 3: Map parameters to Postman requests
    for item in postman_data['item']:
        for sub_item in item.get('item', []):
            if 'request' in sub_item and 'url' in sub_item['request']:
                raw_url = sub_item['request']['url']['raw']
                sub_item['parameters'] = find_parameters(raw_url, req_params)
        for idx, sub_item in enumerate(item['item']):
            sub_item['_index'] = idx

    # Step 4: Render HTML using Jinja2
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template(template_path)
    output = template.render(data=postman_data)

    # Step 5: Write output to HTML
    with open(output_html_path, 'w', encoding='utf-8') as html_file:
        html_file.write(output)

    print(f"HTML file has been generated: {output_html_path}")

# Run the main function
if __name__ == "__main__":
    main()


HTML file has been generated: index.html
